# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [0]:
##### Your Code Here #####

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [1]:
import sys
import numpy as np

class NeuralNetMLP(object):
    """
    Feed forward nueral network / multi-layer perceptron classifier
    
    Parameters
    ----------
    n_hidden : int (default: 30)
        Number of hidden units
    l2 : float (default 0.)
        Lambda value for l2 normalization
    epochs : int (default: 100)
        Number of training epochs
    eta : float (default = 0.001)
        Learning rate
    shuffle : bool (default: True)
        Shuffles the training data every epoch if True
    minibatch_size : int (default : 1)
        Number of training samples per minibatch
    seed : int (default: None)
        Random seed for initalizing weights and shuffling
    
    Attributes
    ----------
    eval_ : dict
        Dictionary collecting the cost, training accuracy,
        and validation accuracy for each epoch during training
    
    """
    
    def __init__(self, n_hidden=30, l2=0.,
                epochs=100, eta=0.001,
                shuffle=True, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size
        
    def _onehot(self, y, n_classes):
        """
        Encode labels into one hot representation
        
        Parameters
        ----------
        y : array, shape = [n_samples]
            Target values
        
        Returns
        -------
        onehot : array, shape = (n_samples, n_labels)
        """
        
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T
    
    def _sigmoid(self, z):
        """
        Compute logistic function (sigmoid)
        """
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))
    
    def _forward(self, X):
        """
        Compute forward propogation step
        """
        # step 1: net input of hidden layer
        z_h = np.dot(X, self.w_h) + self.b_h
        
        # step 2: activation of hidden layer
        a_h = self._sigmoid(z_h)
        
        # step 3: net input of output layer
        z_out = np.dot(a_h, self.w_out) + self.b_out
        
        # step 4: activation layer output
        a_out = self._sigmoid(z_out)
        
        return z_h, a_h, z_out, a_out
    
    def _compute_cost(self, y_enc, output):
        """
        Compute cost function
        
        Parameters
        ----------
        y_enc : array, shape = (n_samples, n_labels)
            one-hot encoded class labels
        output : array, shape = [n_samples, n_output_units]
            Activation of the output layer (forward propoagation)
        Returns
        -------
        cost : float
            Regularized cost
        """
        e = 0.000000001
        L2_term = (self.l2 *
                  (np.sum(self.w_h ** 2.) + 
                  np.sum(self.w_out ** 2.)))
        term1 = -y_enc * (np.log(output + e))
        term2 = (1. - y_enc) * np.log(1. - output + e)
        cost = np.sum(term1 - term2) + L2_term
        return cost
    
    def predict(self, X):
        """
        Predict class labels
        
        Paramters
        ---------
        X : array, shape = [n_samples, n_features]
            Input layer with original features
        
        Returns
        -------
        y_pred : array, shape = [n_samples]
            Predicted class labels
        """
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred
    
    def fit(self, X_train, y_train, X_valid, y_valid):
        """
        Learn weights from training data
        
        Parameters
        ----------
        X_train : array, shape = [n_samples, n_features]
            Input layer with original features
        y_train : array, shape = [n_samples]
            Target class labels
        X_test : array, shape = [n_samples, n_features]
            Sample features for validation during training
        y_test : array, shape = [n_samples]
            Samples test class labels for validation during training
            
        Returns
        -------
        self
        """
        n_output = np.unique(y_train).shape[0]
        n_features = X_train.shape[1]
        
        #######################
        # Weight Initialization
        #######################
        
        # weights for input -> hidden
        self.b_h = np.zeros(self.n_hidden)
        self.w_h = self.random.normal(loc=0.0,
                                     scale=0.1,
                                     size=(n_features,
                                          self.n_hidden))
        
        # weights for hidden -> output
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0,
                                       scale=0.1,
                                       size=(self.n_hidden,
                                       n_output))
        
        epoch_strlen = len(str(self.epochs)) # for progr. format
        self.eval_ = {'cost' : [],
                     'train_acc' : [],
                     'valid_acc' : []}
        
        y_train_enc = self._onehot(y_train, n_output)
        
        # iterate over training epochs
        for i in range(self.epochs):
            
            # iterate over mini batches
            indices = np.arange(X_train.shape[0])
            
            if self.shuffle:
                self.random.shuffle(indices)
            
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                
                # forward propagation
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])
                
                #################
                # Backpropagation
                #################
                
                # [n_samples, n_classlabels]
                sigma_out = a_out - y_train_enc[batch_idx]
                
                # [n_samples, n_hidden]
                sigmoid_derivative_h = a_h * (1. - a_h)
                
                # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_samples, n_hidden]
                sigma_h = (np.dot(sigma_out, self.w_out.T) * sigmoid_derivative_h)
                
                # [n_features, n_samples] dot [n_samples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)
                
                # [n_hidden, n_samples] dot [n_samples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)
                
                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2 * self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h
                
                delta_w_out = grad_w_out + self.l2 * self.w_out
                delta_b_out = grad_b_out # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out
                
            ############
            # Evaluation
            ############

            # evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._compute_cost(y_enc=y_train_enc, output=a_out)
            
            y_train_pred = self.predict(X_train)
            y_valid_pred = self.predict(X_valid)
            
            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) / X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(np.float) / X_valid.shape[0])
            
            sys.stderr.write('\r%0*d/%d | Cost: %.2f | Train/Valid Acc: %.2f%%/%.2f%%'
                            % (epoch_strlen, i+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
            
            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['valid_acc'].append(valid_acc)
            
        return self

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?